<a href="https://colab.research.google.com/github/alexshultz/comfyui-colab/blob/main/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
# Install xFormers for CUDA 12.4 first
!pip install xformers!=0.0.18 --index-url https://download.pytorch.org/whl/cu124
# Then install remaining requirements
!pip install -r requirements.txt

# InsightFace and onnxruntime
!echo -= Installing InsightFace and onnxruntime for IP-Adapter FaceID =-
!pip install insightface onnxruntime-gpu

# Update or install IP-Adapter custom nodes
!echo -= Updating ComfyUI_IPAdapter_plus =-
# If directory exists and is a git repo, update it; otherwise, clone fresh
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd ComfyUI_IPAdapter_plus && git pull || git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git ComfyUI_IPAdapter_plus)
# Install dependencies
!if [ -f /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/requirements.txt ]; then pip install -r /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/requirements.txt; else echo "requirements.txt missing, installing manually" && pip install insightface opencv-python-headless numpy torch torchvision; fi

# Update or install ComfyUI_essentials custom nodes
!echo -= Updating ComfyUI_essentials =-
# If directory exists and is a git repo, update it; otherwise, clone fresh
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd ComfyUI_essentials && git pull || git clone https://github.com/cubiq/ComfyUI_essentials.git ComfyUI_essentials)
# Install dependencies
!if [ -f /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_essentials/requirements.txt ]; then pip install -r /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_essentials/requirements.txt; else echo "No additional requirements for ComfyUI_essentials"; fi


# Update or install Comfy_Dungeon custom nodes
# https://github.com/cubiq/Comfy_Dungeon
!echo -= Updating Comfy_Dungeon =-
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd Comfy_Dungeon && git pull || git clone https://github.com/cubiq/Comfy_Dungeon.git Comfy_Dungeon)
# No requirements.txt specified in README, assuming base ComfyUI deps suffice

# Install huggingface model dowloader
!echo -= huggingface_hub =-
!pip install huggingface_hub


!echo -= Done =-


Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

Great tools in the AP-Adapter package
`https://github.com/cubiq/ComfyUI_IPAdapter_plus?tab=readme-ov-file`

In [ ]:
#@title Environment Setup

import os
import shutil
from huggingface_hub import snapshot_download
import hashlib

# Get Hugging Face token from Colab Secrets
HF_TOKEN = userdata.get('HF_TOKEN')  # Add this to Secrets as 'HF_TOKEN'

OPTIONS = {}
USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /
    from google.colab import drive
    drive.mount('/content/drive')
    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
    !echo -= Updating ComfyUI =-
    !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 --index-url https://download.pytorch.org/whl/cu124
!pip install -r requirements.txt
!pip install huggingface_hub insightface onnxruntime-gpu

# Ensure standard ComfyUI directories exist
base_dir = "/content/drive/MyDrive/ComfyUI/models"
!mkdir -p {base_dir}/checkpoints
!mkdir -p {base_dir}/loras
!mkdir -p {base_dir}/ipadapter
!mkdir -p {base_dir}/vae
!mkdir -p {base_dir}/clip_vision
!mkdir -p {base_dir}/controlnet
!mkdir -p {base_dir}/style_models
!mkdir -p {base_dir}/gligen
!mkdir -p {base_dir}/upscale_models
!mkdir -p {base_dir}/insightface
!mkdir -p {base_dir}/hf_downloads

# Download InsightFace buffalo_l models
!echo -= Downloading InsightFace buffalo_l models =-
!wget -c https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip -O {base_dir}/insightface/buffalo_l.zip
!unzip -o {base_dir}/insightface/buffalo_l.zip -d {base_dir}/insightface/
!rm {base_dir}/insightface/buffalo_l.zip

# Update or install custom nodes
!echo -= Updating ComfyUI_IPAdapter_plus =-
!cd {base_dir}/../custom_nodes && (cd ComfyUI_IPAdapter_plus && git pull || git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git ComfyUI_IPAdapter_plus)
!if [ -f {base_dir}/../custom_nodes/ComfyUI_IPAdapter_plus/requirements.txt ]; then pip install -r {base_dir}/../custom_nodes/ComfyUI_IPAdapter_plus/requirements.txt; else echo "requirements.txt missing, installing manually" && pip install insightface opencv-python-headless numpy torch torchvision onnxruntime-gpu; fi

!echo -= Updating ComfyUI_essentials =-
!cd {base_dir}/../custom_nodes && (cd ComfyUI_essentials && git pull || git clone https://github.com/cubiq/ComfyUI_essentials.git ComfyUI_essentials)
!if [ -f {base_dir}/../custom_nodes/ComfyUI_essentials/requirements.txt ]; then pip install -r {base_dir}/../custom_nodes/ComfyUI_essentials/requirements.txt; else echo "No additional requirements for ComfyUI_essentials"; fi

!echo -= Updating Comfy_Dungeon =-
!cd {base_dir}/../custom_nodes && (cd Comfy_Dungeon && git pull || git clone https://github.com/cubiq/Comfy_Dungeon.git Comfy_Dungeon)

!echo -= Updating comfy_controlnet_preprocessors =-
!cd {base_dir}/../custom_nodes && (cd comfy_controlnet_preprocessors && git pull || git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors.git comfy_controlnet_preprocessors && python install.py)

# Function to sort downloaded files into ComfyUI subdirectories
def sort_hf_files(src_dir, base_dir):
    def get_file_hash(file_path):
        """Calculate SHA256 hash of a file."""
        sha256 = hashlib.sha256()
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                sha256.update(chunk)
        return sha256.hexdigest()

    for root, _, files in os.walk(src_dir):
        for file in files:
            src_path = os.path.join(root, file)
            if file.endswith('.safetensors'):
                if 'ip-adapter' in file.lower():
                    dest_dir = f"{base_dir}/ipadapter"
                elif 'lora' in file.lower():
                    dest_dir = f"{base_dir}/loras"
                elif 'vae' in file.lower():
                    dest_dir = f"{base_dir}/vae"
                elif 'clip-vit' in file.lower() or 'image_encoder' in file.lower():
                    dest_dir = f"{base_dir}/clip_vision"
                elif 'control' in file.lower():
                    dest_dir = f"{base_dir}/controlnet"
                elif 'style' in file.lower():
                    dest_dir = f"{base_dir}/style_models"
                elif 'gligen' in file.lower():
                    dest_dir = f"{base_dir}/gligen"
                else:
                    dest_dir = f"{base_dir}/checkpoints"
                dest_path = os.path.join(dest_dir, file)
                # Copy if file doesn't exist or contents differ
                if not os.path.exists(dest_path):
                    shutil.copy2(src_path, dest_path)
                    print(f"Copied {file} to {dest_dir} (new file)")
                elif get_file_hash(src_path) != get_file_hash(dest_path):
                    shutil.copy2(src_path, dest_path)
                    print(f"Copied {file} to {dest_dir} (updated content)")
                else:
                    print(f"Skipped {file} - already up-to-date in {dest_dir}")
            elif file.endswith(('.pth', '.ckpt', '.bin', '.pt')):
                if 'vae' in file.lower():
                    dest_dir = f"{base_dir}/vae"
                elif 'control' in file.lower():
                    dest_dir = f"{base_dir}/controlnet"
                elif 'style' in file.lower():
                    dest_dir = f"{base_dir}/style_models"
                elif 'gligen' in file.lower():
                    dest_dir = f"{base_dir}/gligen"
                elif 'esrgan' in file.lower() or 'upscale' in file.lower():
                    dest_dir = f"{base_dir}/upscale_models"
                else:
                    dest_dir = f"{base_dir}/checkpoints"
                dest_path = os.path.join(dest_dir, file)
                if not os.path.exists(dest_path):
                    shutil.copy2(src_path, dest_path)
                    print(f"Copied {file} to {dest_dir} (new file)")
                elif get_file_hash(src_path) != get_file_hash(dest_path):
                    shutil.copy2(src_path, dest_path)
                    print(f"Copied {file} to {dest_dir} (updated content)")
                else:
                    print(f"Skipped {file} - already up-to-date in {dest_dir}")

# Download models using snapshot_download
!echo -= Downloading Models =-

# Dreamshaper 8 (checkpoint via wget, as it's from Civitai)
!wget -c https://civitai.com/api/download/models/128713 -O {base_dir}/checkpoints/dreamshaper_8.safetensors

# ProteusV0.3
print("Downloading ProteusV0.3...")
snapshot_download(repo_id="dataautogpt3/ProteusV0.3", local_dir=f"{base_dir}/hf_downloads/ProteusV0.3", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/ProteusV0.3", base_dir)

# dreamshaper-xl-v2-turbo
print("Downloading dreamshaper-xl-v2-turbo...")
snapshot_download(repo_id="Lykon/dreamshaper-xl-v2-turbo", local_dir=f"{base_dir}/hf_downloads/dreamshaper-xl-v2-turbo", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/dreamshaper-xl-v2-turbo", base_dir)

# lcm-lora-sdxl
print("Downloading lcm-lora-sdxl...")
snapshot_download(repo_id="latent-consistency/lcm-lora-sdxl", local_dir=f"{base_dir}/hf_downloads/lcm-lora-sdxl", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/lcm-lora-sdxl", base_dir)

# Stable Diffusion XL base 1.0
print("Downloading SD-XL 1.0-base...")
snapshot_download(repo_id="stabilityai/stable-diffusion-xl-base-1.0", local_dir=f"{base_dir}/hf_downloads/SD-XL_1.0-base", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/SD-XL_1.0-base", base_dir)

# clip_vision_g (SDXL ReVision)
print("Downloading clip_vision_g...")
snapshot_download(repo_id="comfyanonymous/clip_vision_g", local_dir=f"{base_dir}/hf_downloads/clip_vision_g", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/clip_vision_g", base_dir)

# Stable Diffusion v1.5
print("Downloading Stable Diffusion v1.5...")
snapshot_download(repo_id="Comfy-Org/stable-diffusion-v1-5-archive", local_dir=f"{base_dir}/hf_downloads/Stable_Diffusion_v1.5", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/Stable_Diffusion_v1.5", base_dir)

# OrangeMixs
print("Downloading OrangeMixs...")
snapshot_download(repo_id="WarriorMama777/OrangeMixs", local_dir=f"{base_dir}/hf_downloads/OrangeMixs", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/OrangeMixs", base_dir)

# Anything V3 (gated, requires token)
# Uncomment and add token if you have access
# print("Downloading Anything V3...")
# snapshot_download(repo_id="Linaqruf/anything-v3.0", local_dir=f"{base_dir}/hf_downloads/Anything_V3", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"], token="YOUR_HF_TOKEN")
# sort_hf_files(f"{base_dir}/hf_downloads/Anything_V3", base_dir)

# IP-Adapter
print("Downloading IP-Adapter...")
snapshot_download(repo_id="h94/IP-Adapter", local_dir=f"{base_dir}/hf_downloads/IP-Adapter", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/IP-Adapter", base_dir)

# IP-Adapter-FaceID
print("Downloading IP-Adapter-FaceID...")
snapshot_download(repo_id="h94/IP-Adapter-FaceID", local_dir=f"{base_dir}/hf_downloads/IP-Adapter-FaceID", allow_patterns=["*.safetensors", "*.bin"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/IP-Adapter-FaceID", base_dir)

# NSFWmodel
print("Downloading NSFWmodel...")
snapshot_download(repo_id="lexa862/NSFWmodel", local_dir=f"{base_dir}/hf_downloads/NSFWmodel", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/NSFWmodel", base_dir)

# illuminatiDiffusionV1_v11_unCLIP
print("Downloading illuminatiDiffusionV1_v11_unCLIP...")
snapshot_download(repo_id="comfyanonymous/illuminatiDiffusionV1_v11_unCLIP", local_dir=f"{base_dir}/hf_downloads/illuminatiDiffusionV1_v11_unCLIP", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/illuminatiDiffusionV1_v11_unCLIP", base_dir)

# wd-1.5-beta2_unCLIP
print("Downloading wd-1.5-beta2_unCLIP...")
snapshot_download(repo_id="comfyanonymous/wd-1.5-beta2_unCLIP", local_dir=f"{base_dir}/hf_downloads/wd-1.5-beta2_unCLIP", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/wd-1.5-beta2_unCLIP", base_dir)

# sd-vae-ft-mse-original
print("Downloading sd-vae-ft-mse-original...")
snapshot_download(repo_id="stabilityai/sd-vae-ft-mse-original", local_dir=f"{base_dir}/hf_downloads/sd-vae-ft-mse-original", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/sd-vae-ft-mse-original", base_dir)

# Waifu Diffusion v1.4
print("Downloading Waifu Diffusion v1.4...")
snapshot_download(repo_id="hakurei/waifu-diffusion-v1-4", local_dir=f"{base_dir}/hf_downloads/Waifu_Diffusion_v1.4", allow_patterns=["*.safetensors", "*.ckpt"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/Waifu_Diffusion_v1.4", base_dir)

# clip-vit-large-patch14
print("Downloading clip-vit-large-patch14...")
snapshot_download(repo_id="openai/clip-vit-large-patch14", local_dir=f"{base_dir}/hf_downloads/clip-vit-large-patch14", allow_patterns=["*.bin"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/clip-vit-large-patch14", base_dir)

# ControlNet-v1-1_fp16_safetensors (example, adjust as needed)
print("Downloading ControlNet-v1-1_fp16_safetensors...")
snapshot_download(repo_id="comfyanonymous/ControlNet-v1-1_fp16_safetensors", local_dir=f"{base_dir}/hf_downloads/ControlNet-v1-1_fp16_safetensors", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/ControlNet-v1-1_fp16_safetensors", base_dir)

# Control-LoRA Model
print("Downloading Control-LoRA Model...")
snapshot_download(repo_id="stabilityai/control-lora", local_dir=f"{base_dir}/hf_downloads/Control-LoRA_Model", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/Control-LoRA_Model", base_dir)

# GLIGEN
!echo -= Downloading GLIGEN =-
!wget -c https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -O {base_dir}/gligen/gligen_sd14_textbox_pruned_fp16.safetensors

# ESRGAN upscale models
!echo -= Downloading ESRGAN upscale models =-
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -O {base_dir}/upscale_models/RealESRGAN_x4plus.pth
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -O {base_dir}/upscale_models/RealESRGAN_x2.pth
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -O {base_dir}/upscale_models/RealESRGAN_x4.pth

# Verify downloads
!echo -= Verifying Downloads =-
!ls -lh {base_dir}/checkpoints/
!ls -lh {base_dir}/loras/
!ls -lh {base_dir}/ipadapter/
!ls -lh {base_dir}/vae/
!ls -lh {base_dir}/clip_vision/
!ls -lh {base_dir}/controlnet/
!ls -lh {base_dir}/style_models/
!ls -lh {base_dir}/gligen/
!ls -lh {base_dir}/upscale_models/

!echo -= Done =-


### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server